In [1]:
%pip install torch torchvision torchaudio --upgrade -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5

#  The minimal language model

# Lets try with Shahnameh!

In [ ]:
!wget -q https://raw.githubusercontent.com/theheroxx/shahnameh-minimal-LM/refs/heads/master/shahnameh.txt # Get the dataset

In [3]:
# Read the Shahnameh text (UTF-8)
shahnameh = open("shahnameh.txt", encoding='utf-8').read()

In [4]:
import torch
from torch.utils.data import Dataset

sequence_length = 100

def split_input(input_text, sequence_length):
    for i in range(0, len(input_text), sequence_length):
        yield input_text[i : i + sequence_length]

features = list(split_input(shahnameh[:-1], sequence_length))
labels = list(split_input(shahnameh[1:], sequence_length))

class TextDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    def __len__(self):
        return len(self.features)
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

dataset = TextDataset(features, labels)

In [5]:
x, y = dataset[0]
print(x[1:12])
print(y[1:30])

به نام خداو
ه نام خداوند جان و خرد
|کزین 


In [ ]:
class CharTokenizer:
    def __init__(self, sequence_length):
        self.sequence_length = sequence_length
        self.vocab = []
        self.char_to_id = {}
        self.id_to_char = {}
    def adapt(self, texts):
        chars = set()
        for t in texts:
            chars.update(list(t))
        # Reserve 0 for padding and 1 for unknown
        self.vocab = ['<PAD>', '<UNK>'] + sorted(chars)
        self.char_to_id = {c:i for i,c in enumerate(self.vocab)}
        self.id_to_char = {i:c for c,i in self.char_to_id.items()}
    def __call__(self, text):
        ids = [self.char_to_id.get(c, self.char_to_id['<UNK>']) for c in text]
        ids = ids[:self.sequence_length]
        if len(ids) < self.sequence_length:
            ids += [self.char_to_id['<PAD>']] * (self.sequence_length - len(ids))
        return ids
    def get_vocabulary(self):
        return self.vocab
    def vocabulary_size(self):
        return len(self.vocab)

# Build tokenizer from dataset text features
tokenizer = CharTokenizer(sequence_length)
tokenizer.adapt([text for text, _ in dataset])


vocabulary_size = tokenizer.vocabulary_size()
vocabulary_size

50

In [ ]:
tokenizer.get_vocabulary() # full vocabulary

['<PAD>',
 '<UNK>',
 '\n',
 ' ',
 '(',
 ')',
 '|',
 '«',
 '»',
 '،',
 '؟',
 'ء',
 'آ',
 'أ',
 'ؤ',
 'ئ',
 'ا',
 'ب',
 'ت',
 'ث',
 'ج',
 'ح',
 'خ',
 'د',
 'ذ',
 'ر',
 'ز',
 'س',
 'ش',
 'ص',
 'ض',
 'ط',
 'ظ',
 'ع',
 'غ',
 'ف',
 'ق',
 'ل',
 'م',
 'ن',
 'ه',
 'و',
 'ٔ',
 'پ',
 'چ',
 'ژ',
 'ک',
 'گ',
 'ی',
 '\u200c']

In [35]:
from torch.utils.data import DataLoader

class TokenizedDataset(Dataset):
    def __init__(self, features, labels, tokenizer):
        self.features = features
        self.labels = labels
        self.tokenizer = tokenizer
    def __len__(self):
        return len(self.features)
    def __getitem__(self, idx):
        f = torch.tensor(self.tokenizer(self.features[idx]), dtype=torch.long)
        l = torch.tensor(self.tokenizer(self.labels[idx]), dtype=torch.long)
        return f, l

dataset = TokenizedDataset(features, labels, tokenizer)
training_data = DataLoader(dataset, batch_size=64, shuffle=True)

In [36]:
a, b = next(iter(training_data))  # a: [batch, seq_len], b: [batch, seq_len]
a.shape, b.shape

(torch.Size([64, 100]), torch.Size([64, 100]))

In [37]:
import torch.nn as nn

embedding_dim = 256
hidden_dim = 1024

class CharRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim=256, hidden_dim=1024):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    def forward(self, x, hidden=None):
        x = self.embedding(x)
        out, hidden = self.gru(x, hidden)
        out = self.dropout(out)
        logits = self.fc(out)
        return logits, hidden

model = CharRNN(vocabulary_size, embedding_dim, hidden_dim)

In [38]:
print(model)
total_params = sum(p.numel() for p in model.parameters())
print("Total parameters:", total_params)

CharRNN(
  (embedding): Embedding(50, 256)
  (gru): GRU(256, 1024, batch_first=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (fc): Linear(in_features=1024, out_features=50, bias=True)
)
Total parameters: 4002354


In [39]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
epochs = 20

for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    total_tokens = 0
    correct = 0
    for batch_x, batch_y in training_data:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        optimizer.zero_grad()
        logits, _ = model(batch_x)
        loss = criterion(logits.view(-1, vocabulary_size), batch_y.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * batch_x.size(0)
        preds = logits.argmax(dim=-1)
        correct += (preds == batch_y).float().sum().item()
        total_tokens += batch_x.numel()
    print(f"Epoch {epoch+1}/{epochs} loss: {total_loss/len(training_data.dataset):.4f} acc: {correct/total_tokens:.4f}")

# Save the model weights
import os
os.makedirs("/content/drive/MyDrive", exist_ok=True)
torch.save(model.state_dict(), "/content/drive/MyDrive/shahnameh_model1.pt")

Epoch 1/20 loss: 1.6497 acc: 0.5100
Epoch 2/20 loss: 1.3256 acc: 0.5952
Epoch 3/20 loss: 1.2568 acc: 0.6143
Epoch 4/20 loss: 1.2144 acc: 0.6264
Epoch 5/20 loss: 1.1804 acc: 0.6361
Epoch 6/20 loss: 1.1523 acc: 0.6442
Epoch 7/20 loss: 1.1274 acc: 0.6512
Epoch 8/20 loss: 1.1060 acc: 0.6576
Epoch 9/20 loss: 1.0872 acc: 0.6628
Epoch 10/20 loss: 1.0707 acc: 0.6675
Epoch 11/20 loss: 1.0561 acc: 0.6715
Epoch 12/20 loss: 1.0440 acc: 0.6751
Epoch 13/20 loss: 1.0338 acc: 0.6779
Epoch 14/20 loss: 1.0275 acc: 0.6799
Epoch 15/20 loss: 1.0208 acc: 0.6813
Epoch 16/20 loss: 1.0198 acc: 0.6819
Epoch 17/20 loss: 1.0163 acc: 0.6826
Epoch 18/20 loss: 1.0165 acc: 0.6826
Epoch 19/20 loss: 1.0182 acc: 0.6819
Epoch 20/20 loss: 1.0212 acc: 0.6810


In [40]:
import pickle

with open("/content/drive/MyDrive/shahnameh_tokenizer1.pkl", "wb") as f:
    pickle.dump(tokenizer.get_vocabulary(), f)

--------------------

# Initial test for making a generation model, a better code with GUI is available in the files

In [46]:
# Create a generation-ready model (same architecture) and helper for stepwise generation
generation_model = CharRNN(vocabulary_size, embedding_dim, hidden_dim)
generation_model.load_state_dict(model.state_dict())
generation_model.to(device)
generation_model.eval()

# Step function: given a token id and previous hidden, return logits for next token and new hidden
import torch

def generation_step(token_id, hidden):
    x = torch.tensor([[token_id]], dtype=torch.long, device=device)  # batch=1, seq_len=1
    with torch.no_grad():
        logits, hidden = generation_model(x, hidden)
    return logits[:, -1, :], hidden

# Save the generation model weights
torch.save(generation_model.state_dict(), "/content/drive/MyDrive/shahnameh_generation_model1.pt")

In [48]:
tokens = tokenizer.get_vocabulary()
char_to_id = {c:i for i,c in enumerate(tokens)}
id_to_char = {i:c for c,i in char_to_id.items()}

prompt = """
ضحاک
"""

In [49]:
# Prepare prompt and initial hidden state
input_ids = [char_to_id.get(c, char_to_id['<UNK>']) for c in prompt]
hidden = torch.zeros(1, 1, hidden_dim, device=device)

# Feed the prompt into the model to prime the hidden state
with torch.no_grad():
    for token_id in input_ids:
        logits, hidden = generation_step(token_id, hidden)
    # 'logits' now holds the distribution for the next char

In [ ]:
generated_ids = []
max_length = 250 # maximum generation tokens
for i in range(max_length):
    next_token = int(torch.argmax(logits, dim=-1).item()) # argmax sets the most possible word for each timestep to 1 and others to 0
    generated_ids.append(next_token)
    logits, hidden = generation_step(next_token, hidden)

In [52]:
output = "".join([id_to_char[token_id] for token_id in generated_ids])
print(prompt + output)


ضحاک
|همی باش تا جاودان اندکی
|بپرسید زو شهریار جهان
|به پیش سپاه اندر آمد به جوش
|ز هر سو برانگیخت اسفندیار
|برفتند با نامداران را
|بدو گفت بهرام کای شهریار
|به مردی و نیک و بد و نیک رای
|به مردی و گردی و گردنکشان
|کنون چون بود رنج و آزار اوی
|برو تا برآ
